In [1439]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import classification_report
from catboost import CatBoostClassifier
from sklearn.ensemble import HistGradientBoostingClassifier


In [1440]:
data = pd.read_csv('../dataset/preprocessed_train_data')
data

,passenger_id,age,room_service,food_court,shopping_mall,spa,v_r_deck,name,transported,Europa,...,deck_D,deck_E,deck_F,deck_G,deck_T,deck_unknown,side_S,side_unknown,v_i_p_True,v_i_p_unknown
0,0001_01,39.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0002_01,24.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1,0,...,0,0,1,0,0,0,1,0,0,0
2,0003_01,58.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0,1,...,0,0,0,0,0,0,1,0,1,0
3,0003_02,33.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0,1,...,0,0,0,0,0,0,1,0,0,0
4,0004_01,16.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,41.0,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,0,1,...,0,0,0,0,0,0,0,0,1,0
8689,9278_01,18.0,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,0,0,...,0,0,0,1,0,0,1,0,0,0
8690,9279_01,26.0,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,1,0,...,0,0,0,1,0,0,1,0,0,0
8691,9280_01,32.0,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,0,1,...,0,1,0,0,0,0,1,0,0,0


In [1441]:
data = data.drop(columns=['passenger_id', 'name'])
data.columns = data.columns.str.lower()

In [1442]:
data

,age,room_service,food_court,shopping_mall,spa,v_r_deck,transported,europa,mars,unknown_planet,...,deck_d,deck_e,deck_f,deck_g,deck_t,deck_unknown,side_s,side_unknown,v_i_p_true,v_i_p_unknown
0,39.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,...,0,0,1,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
8689,18.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,0
8690,26.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,...,0,0,0,1,0,0,1,0,0,0
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,0,1,0,0,...,0,1,0,0,0,0,1,0,0,0


In [1443]:
data['total_spent'] = data[['room_service', 'food_court', 'shopping_mall', 'spa', 'v_r_deck']].sum(axis=1)
data['room_service_ratio'] = data['room_service'] / data['total_spent']
data['room_service_ratio'] = data['room_service_ratio'].fillna(data['room_service_ratio'].median())

In [1444]:
ratio_features = ['room_service', 'food_court', 'shopping_mall', 'spa', 'v_r_deck']
def spent_ratio(dataset, new_feature_name, first_feature, second_feature):
    dataset[new_feature_name] = dataset[first_feature] / dataset[second_feature]
    dataset[new_feature_name] = dataset[new_feature_name].fillna(dataset[new_feature_name].median())

In [1445]:
for i in range(len(ratio_features) - 1):
    spent_ratio(data, f'{ratio_features[i]}_ratio', ratio_features[i], 'total_spent')

In [1446]:
data

,age,room_service,food_court,shopping_mall,spa,v_r_deck,transported,europa,mars,unknown_planet,...,deck_unknown,side_s,side_unknown,v_i_p_true,v_i_p_unknown,total_spent,room_service_ratio,food_court_ratio,shopping_mall_ratio,spa_ratio
0,39.0,0.0,0.0,0.0,0.0,0.0,0,1,0,0,...,0,0,0,0,0,0.0,0.005607,0.011891,0.003574,0.011036
1,24.0,109.0,9.0,25.0,549.0,44.0,1,0,0,0,...,0,1,0,0,0,736.0,0.148098,0.012228,0.033967,0.745924
2,58.0,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,...,0,1,0,1,0,10383.0,0.004141,0.344409,0.000000,0.646730
3,33.0,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,...,0,1,0,0,0,5176.0,0.000000,0.247875,0.071677,0.643161
4,16.0,303.0,70.0,151.0,565.0,2.0,1,0,0,0,...,0,1,0,0,0,1091.0,0.277727,0.064161,0.138405,0.517874
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,41.0,0.0,6819.0,0.0,1643.0,74.0,0,1,0,0,...,0,0,0,1,0,8536.0,0.000000,0.798852,0.000000,0.192479
8689,18.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,...,0,1,0,0,0,0.0,0.005607,0.011891,0.003574,0.011036
8690,26.0,0.0,0.0,1872.0,1.0,0.0,1,0,0,0,...,0,1,0,0,0,1873.0,0.000000,0.000000,0.999466,0.000534
8691,32.0,0.0,1049.0,0.0,353.0,3235.0,0,1,0,0,...,0,1,0,0,0,4637.0,0.000000,0.226224,0.000000,0.076127


In [1447]:
X = data.drop(columns='transported')
y = data['transported']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

imputer = SimpleImputer(strategy='mean')
X_train = imputer.fit_transform(X_train)
X_test = imputer.transform(X_test)

In [1448]:
from sklearn.preprocessing import Normalizer

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [1449]:
X_train = pd.DataFrame(X_train, columns=X.columns)
X_test = pd.DataFrame(X_test, columns=X.columns)
X_train

,age,room_service,food_court,shopping_mall,spa,v_r_deck,europa,mars,unknown_planet,cryo_sleep_true,...,deck_unknown,side_s,side_unknown,v_i_p_true,v_i_p_unknown,total_spent,room_service_ratio,food_court_ratio,shopping_mall_ratio,spa_ratio
0,-1.996149,-0.336340,-0.279076,-0.305732,-0.267311,-0.264632,-0.567054,-0.506553,-0.157822,-0.734286,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,-0.517737,-0.452702,-0.441896,-0.412507,-0.411815
1,-0.811703,-0.336340,-0.279076,-0.305732,-0.267311,-0.264632,-0.567054,-0.506553,-0.157822,1.361866,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,-0.517737,-0.452702,-0.441896,-0.412507,-0.411815
2,0.442416,-0.336340,-0.279076,-0.305732,-0.267311,-0.264632,-0.567054,-0.506553,-0.157822,-0.734286,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,-0.517737,-0.452702,-0.441896,-0.412507,-0.411815
3,-0.184643,-0.336340,-0.279076,-0.305732,-0.267311,-0.264632,1.763499,-0.506553,-0.157822,1.361866,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,-0.517737,-0.452702,-0.441896,-0.412507,-0.411815
4,-1.090396,-0.336340,-0.279076,-0.195161,-0.266427,4.423937,-0.567054,-0.506553,-0.157822,-0.734286,...,-0.153951,-0.981477,-0.153951,-0.152969,-0.152476,1.372395,-0.472643,-0.486236,-0.381072,-0.455318
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6949,0.999803,-0.336340,0.948614,-0.305732,-0.267311,1.239317,1.763499,-0.506553,-0.157822,-0.734286,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,0.788080,-0.472643,1.531605,-0.426643,-0.456088
6950,0.651436,-0.063209,-0.151073,-0.305732,-0.170012,0.076056,-0.567054,-0.506553,-0.157822,-0.734286,...,-0.153951,-0.981477,-0.153951,-0.152969,-0.152476,-0.201989,0.275427,0.383844,-0.426643,0.051136
6951,1.139150,-0.334848,-0.274662,-0.202532,0.274907,-0.264632,-0.567054,-0.506553,-0.157822,-0.734286,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,-0.272034,-0.467390,-0.447680,-0.099445,3.176354
6952,-0.323990,-0.336340,-0.279076,-0.305732,-0.267311,-0.264632,-0.567054,-0.506553,-0.157822,1.361866,...,-0.153951,1.018872,-0.153951,-0.152969,-0.152476,-0.517737,-0.452702,-0.441896,-0.412507,-0.411815


In [1450]:
test_data = pd.read_csv('../dataset/test_data_svm.csv')
test_data.columns = test_data.columns.str.lower()
test_data = test_data.drop(columns='has_family')
passenger_id = test_data['passenger_id']
test_data = test_data.drop(columns='passenger_id')

In [1451]:
test_data['total_spent'] = test_data[['room_service', 'food_court', 'shopping_mall', 'spa', 'v_r_deck']].sum(axis=1)

In [1452]:
for i in range(len(ratio_features) - 1):
    spent_ratio(test_data, f'{ratio_features[i]}_ratio', ratio_features[i], 'total_spent')

In [1453]:
test_data

,age,room_service,food_court,shopping_mall,spa,v_r_deck,europa,mars,unknown_planet,cryo_sleep_true,...,deck_unknown,side_s,side_unknown,v_i_p_true,v_i_p_unknown,total_spent,room_service_ratio,food_court_ratio,shopping_mall_ratio,spa_ratio
0,27.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,...,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,0.000791
1,19.0,109.0,9.0,0.0,2823.0,0.0,0,0,0,0,...,0,1,0,0,0,2941.0,0.037062,0.003060,0.000000,0.959878
2,31.0,43.0,0.0,0.0,0.0,0.0,1,0,0,1,...,0,1,0,0,0,43.0,1.000000,0.000000,0.000000,0.000000
3,38.0,0.0,6652.0,0.0,181.0,585.0,1,0,0,0,...,0,1,0,0,0,7418.0,0.000000,0.896738,0.000000,0.024400
4,20.0,303.0,0.0,635.0,0.0,0.0,0,0,0,0,...,0,1,0,0,0,938.0,0.323028,0.000000,0.676972,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,34.0,0.0,0.0,0.0,0.0,0.0,0,0,0,1,...,0,1,0,0,0,0.0,0.000000,0.000000,0.000000,0.000791
4273,42.0,0.0,847.0,17.0,10.0,144.0,0,0,0,0,...,1,0,1,0,0,1018.0,0.000000,0.832024,0.016699,0.009823
4274,26.0,375.0,0.0,0.0,0.0,0.0,0,1,0,1,...,0,0,0,0,0,375.0,1.000000,0.000000,0.000000,0.000000
4275,26.0,0.0,2680.0,0.0,0.0,523.0,1,0,0,0,...,0,0,0,0,0,3203.0,0.000000,0.836716,0.000000,0.000000


In [1454]:
model = RandomForestClassifier(
    n_estimators=500,
    max_depth=6,
    random_state=42,
)

model.fit(X_train, y_train)

,n_estimators,500
,criterion,'gini'
,max_depth,6
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [1455]:
y_pred = model.predict(X_test)

In [1456]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.77      0.79       863
           1       0.78      0.83      0.80       876

    accuracy                           0.80      1739
   macro avg       0.80      0.80      0.80      1739
weighted avg       0.80      0.80      0.80      1739



In [1457]:
predictions = model.predict(test_data)

In [1458]:
submission = pd.DataFrame({
    'PassengerId': passenger_id,
    'Transported': predictions.astype(bool)
})

submission.to_csv('submission.csv', index=False)
print("✅ Submission файлът е готов: submission.csv")

✅ Submission файлът е готов: submission.csv


In [1459]:
from sklearn.model_selection import cross_val_score
cross_val_score(model, X, y, cv=5, scoring='accuracy').mean()

np.float64(0.7942051666533217)

In [1460]:
# след като моделът е обучен
importances = model.feature_importances_
features = X_train.columns

# таблица с важностите
feat_imp = pd.DataFrame({'Feature': features, 'Importance': importances})
feat_imp = feat_imp.sort_values(by='Importance', ascending=False)
print(feat_imp)

                 Feature  Importance
29      food_court_ratio    0.138428
27           total_spent    0.135251
9        cryo_sleep_true    0.114575
31             spa_ratio    0.082589
28    room_service_ratio    0.074814
4                    spa    0.063561
5               v_r_deck    0.058494
30   shopping_mall_ratio    0.056876
1           room_service    0.056217
2             food_court    0.048385
3          shopping_mall    0.037141
6                 europa    0.025997
20                deck_g    0.025075
11                   num    0.014326
19                deck_f    0.012742
0                    age    0.012513
7                   mars    0.009156
15                deck_b    0.007274
18                deck_e    0.006419
23                side_s    0.005852
16                deck_c    0.005505
13    planet_trappist-1e    0.002747
12  planet_pso j318.5-22    0.001323
17                deck_d    0.000896
26         v_i_p_unknown    0.000771
8         unknown_planet    0.000729
1